In [1]:
import requests      # Для запросов по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами

In [7]:
import json

def get_page_trud(page=0):
  """Gets a page of vacancies from the HeadHunter API.

  Args:
    page: The page number.

  Returns:
    A JSON object containing the vacancies.
  """

  params = {
        'offset': page,
#         'limit': 100
    }
    
  req = requests.get("http://opendata.trudvsem.ru/api/v1/vacancies", params)
    
  data = req.content.decode()
  req.close()

  return data

trud_jobs_objs = []

for page in range(0, 100):
  jobs_obj = json.loads(get_page_trud(page))
  trud_jobs_objs.append(jobs_obj)
#   print(jobs_obj)


#   time.sleep(0.25)

print('finished the search')
len(trud_jobs_objs)

finished the search


100

In [4]:
import re

def remove_html_tags(text):
  """Removes HTML tags from a string.

  Args:
    text: A string or bytes-like object.

  Returns:
    A string or bytes-like object with the HTML tags removed.
  """
  if isinstance(text, str):
    return re.sub(r'[0-9]|[a-z]|[\n]|[-—]|[\<\>\/]|[.;&✓]|[nbsp]+', "", text)
  elif isinstance(text, bytes):
    return re.sub(br"<[^>]*>", b"", text)
  else:
    raise TypeError("The text argument must be a string or bytes-like object.")

In [20]:
trud_datas = []
for i in range(len(trud_jobs_objs)):
  if 'results' in trud_jobs_objs[i]:
    for s in range(100):
      trud_datas.append(trud_jobs_objs[i]['results']['vacancies'][s]['vacancy'])
      print(f"{trud_jobs_objs[i]['results']['vacancies'][s]['vacancy']['job-name']} added")

подсобный рабочий added
Системный администратор added
Кладовщик-комплектовщик added
оператор салатной линии added
Водитель автомобиля added
Преподаватель-организатор added
Администратор added
Корреспондент added
подсобный рабочий added
Телеоператор added
Электромонтер added
Продавец продовольственных товаров added
секретарь- делопроизводитель added
Монтажник по подключению интернета added
Уборщик производственных и служебных помещений added
Медицинская сестра по массажу added
Аппаратчик смешивания added
Кладовщик added
Столяр-плотник added
Медицинская сестра added
Токарь added
Фрезеровщик added
Мойщик посуды added
Воспитатель детского сада added
Музыкальный руководитель в детский сад added
Медицинская сестра, медицинский брат added
Водитель погрузчика added
Грузчик added
Менеджер по работе с клиентами B2B added
Ведущий Архитектор-реставратор added
Кладовщик added
Водитель added
Кладовщик грузчик added
Складской работник added
Пекарь added
Повар-кассир / продавец added
Водитель автобето

In [21]:
len(trud_datas)

10000

In [68]:
trud_datas[8684]

{'id': 'd0975c66-a132-11ec-b31b-550ed7335bbe',
 'source': 'Работодатель',
 'company': {'companycode': '1170917002167',
  'email': 'ooodorogi17@mail.ru',
  'hr-agency': False,
  'inn': '0917032758',
  'kpp': '091701001',
  'name': 'Общество с ограниченной ответственностью "Дороги"',
  'ogrn': '1170917002167',
  'phone': '+7(8782)210326',
  'url': 'https://trudvsem.ru/company/1170917002167'},
 'creation-date': '2022-03-11',
 'salary': 'от 40000',
 'salary_min': 40000,
 'salary_max': 0,
 'job-name': 'Дорожный рабочий',
 'vac_url': 'https://trudvsem.ru/vacancy/card/1170917002167/d0975c66-a132-11ec-b31b-550ed7335bbe',
 'employment': 'Полная занятость',
 'schedule': 'Полный рабочий день',
 'duty': '<p>ООО " Дороги" требуется дорожные рабочие в Хабезском районе.</p>',
 'category': {'specialisation': 'Строительство, ремонт, стройматериалы, недвижимость'},
 'requirement': {'education': 'Не указано', 'experience': 0},
 'social_protected': '',
 'contact_list': [{'contact_type': 'Телефон', 'contac

In [84]:
def fit_trudsvem_data_to_dataset(vacancy):
  """Gets the desired fields from a vacancy.

  Args:
    vacancy: A vacancy.

  Returns:
    A dictionary containing the values of the desired fields.
  """

  results = {}
  results["custom_position"] = vacancy["job-name"]
  results["schedule"] = vacancy["schedule"]
  results["salary_from"] = vacancy["salary_min"]
  results["salary_to"] = vacancy["salary_max"]
  results["education_name"] = vacancy["requirement"]["education"]
  results["work_skills"] = vacancy.get("duty", None)
  if vacancy.get('region'):
      results["region_code"] = vacancy['region']['region_code'][:2]
      results["region_name"] = vacancy['region']['name']
  
  
  return results

In [85]:
trud_to_save_1 = []
for job in trud_datas:
    trud_to_save_1.append(fit_trudsvem_data_to_dataset(job))

In [86]:
len(trud_to_save_1)

10000

In [87]:
res = pd.DataFrame(trud_to_save_1)

In [88]:
res

,custom_position,schedule,salary_from,salary_to,education_name,work_skills,region_code,region_name
0,подсобный рабочий,Полный рабочий день,45000,0,Не указано,"<p>обрыв листа, сбор, сортировка и затаривание...",47,Ленинградская область
1,Системный администратор,Полный рабочий день,40000,0,Не указано,<p>Поддержка пользователей и ИТ инфраструктуры...,48,Липецкая область
2,Кладовщик-комплектовщик,Полный рабочий день,45000,55000,Не указано,<ul>\n<li>Подготовка (сбор) ТМЦ подлежащих отг...,78,Город Санкт-Петербург
3,оператор салатной линии,Полный рабочий день,45000,0,Не указано,"<p>оператор по выращиванию салата, посадка, ух...",47,Ленинградская область
4,Водитель автомобиля,Полный рабочий день,21000,30000,Среднее,<p>Доставка товара в торговые точки города Тих...,23,Краснодарский край
...,...,...,...,...,...,...,...,...
9995,Антенщик-мачтовик 5 разряда,Полный рабочий день,33900,0,Среднее,Осуществляет оперативно-техническое обслуживан...,10,Республика Карелия
9996,Электромонтер по ремонту и обслуживанию электр...,Неполный рабочий день,7900,0,Среднее профессиональное,"Участие в текущем обслуживании, техническом ос...",10,Республика Карелия
9997,Инженер средств радио и телевидения,Полный рабочий день,51600,0,Высшее,"Участие в текущем обслуживании, техническом ос...",10,Республика Карелия
9998,Ведущий инженер средств радио и телевидения,Полный рабочий день,57000,0,Высшее,"Руководство и участие в текущем обслуживании, ...",10,Республика Карелия


In [89]:
res["work_skills"] = res["work_skills"].astype(str)

In [90]:
res["work_skills"] = res["work_skills"].apply(remove_html_tags)

In [91]:
res

,custom_position,schedule,salary_from,salary_to,education_name,work_skills,region_code,region_name
0,подсобный рабочий,Полный рабочий день,45000,0,Не указано,"обрыв листа, сбор, сортировка и затаривание ов...",47,Ленинградская область
1,Системный администратор,Полный рабочий день,40000,0,Не указано,Поддержка пользователей и ИТ инфраструктуры пр...,48,Липецкая область
2,Кладовщик-комплектовщик,Полный рабочий день,45000,55000,Не указано,"Подготовка (сбор) ТМЦ подлежащих отгрузке, офо...",78,Город Санкт-Петербург
3,оператор салатной линии,Полный рабочий день,45000,0,Не указано,"оператор по выращиванию салата, посадка, уход ...",47,Ленинградская область
4,Водитель автомобиля,Полный рабочий день,21000,30000,Среднее,Доставка товара в торговые точки города Тихоре...,23,Краснодарский край
...,...,...,...,...,...,...,...,...
9995,Антенщик-мачтовик 5 разряда,Полный рабочий день,33900,0,Среднее,Осуществляет оперативнотехническое обслуживани...,10,Республика Карелия
9996,Электромонтер по ремонту и обслуживанию электр...,Неполный рабочий день,7900,0,Среднее профессиональное,"Участие в текущем обслуживании, техническом ос...",10,Республика Карелия
9997,Инженер средств радио и телевидения,Полный рабочий день,51600,0,Высшее,"Участие в текущем обслуживании, техническом ос...",10,Республика Карелия
9998,Ведущий инженер средств радио и телевидения,Полный рабочий день,57000,0,Высшее,"Руководство и участие в текущем обслуживании, ...",10,Республика Карелия
